# Inicialização

In [3]:
import logging

import numpy as np
import pandas as pd

import torch
import transformers

ModuleNotFoundError: No module named 'torch'

**Note:**  
Needs installation of PyTorch  
`conda install pytorch torchvision -c pytorch`

Needs installation of Transformers for BERT  
`pip install transformers`

# Carregamento de dados

Carregue os dados de texto do arquivo 'imdb_reviews_small.tsv'.

É um arquivo de valores separados por tabulação (TSV), o que significa que cada um dos campos é separado por tabulações (em vez de separados por vírgulas, como você viu em outras tarefas no Practicum).

In [2]:
data = pd.read_csv('/datasets/imdb_reviews_small.tsv', sep='\t')

# Tokenizador BERT

Criando o tokenizador BERT a partir de um modelo pré-treinado chamado de 'bert-base-uncased' em transformadores. Você pode rapidamente ter uma visão geral [aqui](https://huggingface.co/transformers/pretrained_models.html) e para mais detalhes, você pode ler [aqui](https://huggingface.co/bert-base-uncased).

In [3]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

Há um exemplo de obtenção de tokens para um determinado texto único.
Você pode usá-lo para processar todos os dados que carregou acima. Como já existem muitos textos e é provável que você os processe em um loop, os comprimentos mínimo/máximo de vetores podem ser calculados de duas maneiras: dentro de um ciclo ou após um ciclo.
No último caso, vetores de identificadores numéricos de tokens (ids) e máscaras de atenção (attention_mask) precisam ser armazenados em duas listas separadas. Eles podem ser chamados de **ids_list** e **attention_mask_list**, respectivamente. O primeiro caso nos permite evitar a construção dessas listas, a menos que você queira usá-las para outra finalidade, como, por exemplo, na propagação em um modelo BERT. Não é necessário fazer isto nesta tarefa, mas será necessário no projeto.
Dado o exposto, você pode querer combinar as duas maneiras para calcular os comprimentos mínimo/máximo de vetores para tokens e máscaras de atenção e manter o resultado do tokenizador para processamento adicional. Por favor, tenha em mente que não faz muito sentido manter vetores com mais de 512 elementos, pois este é o comprimento máximo de vetores que o BERT pode aceitar.

In [4]:
# textos para tokens
text = 'It is very handy to use transformers' #(É muito prático usar transformadores)

# adicionando este truque para suprimir avisos de saídas longas
# normalmente não precisamos, mas neste caso vamos querer explorar
# qual é o comprimento máximo de IDs para nosso conjunto de resenhas# portanto, não truncamos a saída (ids) no max_length
# com os parâmetros max_length=max_length e truncation=True
logging.getLogger("transformers.tokenization_utils").setLevel(logging.ERROR)
        
ids = tokenizer.encode(text.lower(), add_special_tokens=True)

# preenchimento (anexando zeros ao vetor para tornar seu comprimento igual a n)
n = 512
padded = np.array(ids[:n] + [0]*(n - len(ids)))

# criando a máscara de atenção para distinguir os tokens nos quais estamos interessados
attention_mask = np.where(padded != 0, 1, 0)

In [5]:
print(ids)

[101, 2009, 2003, 2200, 18801, 2000, 2224, 19081, 102]


In [6]:
print(padded)

[  101  2009  2003  2200 18801  2000  2224 19081   102     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [7]:
print(attention_mask)

[1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

Componha seu código para tokenizar os dados de texto carregados.

In [10]:
def tokenize_with_bert(texts):
    
    ids_list = []
    attention_mask_list = []

    min_tokenized_text_length = 1e7
    max_tokenized_text_length = 0
    
    n = 512
    
    for text in texts:
        ids = tokenizer.encode(text.lower(), add_special_tokens=True)
        text_length = len(ids)
        if text_length < min_tokenized_text_length:
            min_tokenized_text_length = min_tokenized_text_length
        if text_length > max_tokenized_text_length:
            max_tokenized_text_length = max_tokenized_text_length
        
        ids = tokenizer.encode(text.lower(), add_special_tokens=True, max_length=512, truncation=True)
        padded = np.array(ids[:n] + [0]*(n - len(ids)))
        attention_mask = np.where(padded != 0, 1, 0)
        
        ids_list.append(ids)
        attention_mask_list.append(attention_mask)
        
        
        
    print(f'O comprimento mínimo dos vetores: {min_tokenized_text_length}')
    print(f'O comprimento máximo dos vetores: {max_tokenized_text_length}')        
        
    return ids_list, attention_mask_list

Execute o tokenizador para todos os dados. Pode levar algum tempo como 

In [11]:
ids_list, attention_mask_list = tokenize_with_bert(texts=data['review'])

O comprimento mínimo dos vetores: 10000000.0
O comprimento máximo dos vetores: 0
